<a id="top"></a>
# Saftey Gear Detection Demo

## Check for latest version
<br><div class=danger><b>Important: Before proceeding, please run the following cell to ensure that you are running the most recent version of this sample.</b></div>

In [ ]:
# Show the current status of this and all documents with ability to update
from qarpo.catalog import DemoCatalog
import os
status = DemoCatalog(os.getcwd(), "Demo").ShowRepositoryControls()

## Introduction

This sample application demonstrates how a smart video IoT solution may be created using Intel® hardware and software tools to perform safety gear detection.  This solution detects any number of objects within a video frame looking specifically for people, safety vests, and hardhats. 

The results for each frame are stored in a text file that is later read by a second pass to annotate the input video with boxes around detected objects with a label and probability value.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Video and image input is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of the resulting bounding boxes in the output
  - Uses the [Async API](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_Intro_to_Performance.html) feature of the Inference Engine (see [Object Detection Sample](../object-detection-python/object_detection.ipynb) for more details)
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) and [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)


## Safety gear detection application
The safety gear detection application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on an input video to locate people, safety vests, and hardhats within each frame.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Use the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) to create the model IR files in the necessary precisions
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the safety gear detection application configures itself by parsing the command line arguments and reading the specified labels file.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs the three phases described in the following sections.
#### Phase 1:  Pre-processing
The frames from the specified input video are read and pre-processed to have the frame data ready for input into inference.  Each frame is resized and its channels are transposed to match the input of the input requirements of the inference model.  The resulting blocks of frame data are written to the binary file `tmp/processed_vid.bin`.

#### Phase 2:  Inference
The binary file `tmp/processed_vid.bin` is read for input to inference to detect the specific objects: people, safety vests, and hardhats.  For each detected object within a frame, a message is written as a line containing the location, label, etc. to the output text file.

#### Phase 3:  Post-processing
After the safety gear detection application has finished running inference, a second executable is run to annotate the input video with the results from the output text file.  The final results is an output video with boxes drawn around detected objects with each box appearing with a label and probability value.

In this sample, we provide three performance numbers for each architecture. However, they are not hard limits on the solution's performance. It is important to understand that for any application, you may want to combine preprocessing, inference, and post-processing, as opposed to separating them as done here. Combined preprocessing has several advantages:
- If inference is run asynchronously on the accelerator, the rest of the system is available for parallel tasks for capturing input, preprocessing future frames, or post-processing past frames.
- The inference application pipeline has better data locality, allowing for the reuse of data in caches such as in the application's memory or in the hard drive's cache. 

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and and Intel® Arria® 10 FPGA.  After inference on the input is completed, the output is stored in the appropriate `results/<JobNum>/` directory.  The results are then viewed within this Jupyter* Notebook using the `videoHTML` video playback utility.

The application and inference code for this sample is already implemented in the files: 
- First pass, detection using inference: [`safety_gear_detection.py`](./safety_gear_detection.py)
- Second pass, annotation using detection results: [`ROI_writer.cpp`](./ROI_writer.cpp)

The following sections will guide you through configuring and running the safety gear detection demo.

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.).  For this sample, the Caffe* model is supplied in the directory `/data/reference-sample-data/safety-gear-detection/` which contains the model files:
- **worker_safety_mobilenet.prototxt** - The deployed [Protocol Buffer](https://developers.google.com/protocol-buffers) file that describes the network architecture to run inference
- **worker_safety_mobilenet.caffemodel** - Binary containing trained weights

These files will need to be optimized using the Model Optimizer to create the necessary IR files.  We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the two most common precisions, FP32 and FP16.

For this model, we will run the Model Optimizer using the format:
```bash
/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model <path_to_caffemodel> \
    --model_name <name> \
    --data_type <data_precision> \
    -o <path_to_output_directory> 
```

The input arguments are as follows:
- **--input_model** : The model's input *.caffemodel* file  (the *.prototxt* with the same base name will be automatically found, otherwise the `--input_proto` argument would need to be specified)
- **--model_name** : Output model name to use
- **--data_type** : The model's data type and precision (e.g. FP16, FP32, INT8, etc.)
- **--o** : Output directory where to store the generated IR model files

For converting our model, the complete command will look like the following:
```bash
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model /data/reference-sample-data/safety-gear-detection/worker_safety_mobilenet.caffemodel \
    --model_name mobilenet-ssd \
    --data_type <data_precision> \
    -o models/mobilenet-ssd/<data_precision>
```
We will run the command twice, first with <*data_precision*> set to `FP16` and then `FP32` to get all the IR files we will need to run inference on different devices.

<br><div class=note><i><b>Note: </b>More information on how to use Model Optimizer to convert Caffe* models may be found at:[Converting a Caffe* Model](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_Caffe.html)</i></div>

Run the following cell to use the Model Optimizer to create the `FP16` and `FP32` model IR files:

In [ ]:
# Create FP16 IR files
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model /data/reference-sample-data/safety-gear-detection/worker_safety_mobilenet.caffemodel \
--model_name mobilenet-ssd \
--data_type FP16 \
-o models/mobilenet-ssd/FP16 

# Create FP32 IR files
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model /data/reference-sample-data/safety-gear-detection/worker_safety_mobilenet.caffemodel \
--model_name mobilenet-ssd \
--data_type FP32 \
-o models/mobilenet-ssd/FP32

# find all resulting IR files
!echo "\nAll IR files that were created:"
!find ./models -name "*.xml" -o -name "*.bin" 

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning of a line is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The above command will also work in a terminal (with the '!' removed).</i></div>

As shown above from the output of the last `!find...` command, the required sets of IR model files (`*.xml` and `*.bin`) have been created.  

### Optional exercise: View input without inference

If you are curious to see the input video, run the following cell to view the original video stream used for inference and this demo.

In [ ]:
from qarpo.demoutils import videoHTML
!ln -sfn /data data
videoHTML('Saftey Gear Detection Video', ['./data/reference-sample-data/safety-gear-detection/Safety_Full_Hat_and_Vest.mp4'])

### Run the demo

Running the next cell will display an interactive user interface allowing you to submit jobs to run the demo on multiple edge compute nodes selected by hardware devices, view the output of each job, and compare performance results across jobs.

To run a job:
1. Select the desired option in the **Target node** list
2. Select the desired device in the **Target architecture** list
3. Click the **Submit** button

After the **Submit** button is clicked, a tab will appear for the new job with a label in the format "*status*: *JobID*".  Once the job status appears as "Done", the **Display output** button may be clicked to view the output for the job.

After one or more jobs are done, the performance results for each job may be plotted by clicking the **Plot results** button.  Results for each job will be potted as bar graphs for **inference time** and **frames per second**.  Lower values mean better performance for **inference time** and higher values mean better performance for **frames per second**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

Run the next cell to begin the interactive demo.

In [ ]:
# import necessary libraries
import json
import qarpo

# load job configurations for demo
with open('job_config.json') as json_file:  
    data = json.load(json_file)
    
# create and run the user job interface
job_interface = qarpo.Interface(data)
job_interface.displayUI()

## Next steps
- The complete [Saftey Gear Detection Sample](../../developer_samples/safety-gear-detection-python/safety_gear_detection.ipynb) for this demo
- [Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) -  tutorials on using and creating Jupyter* Notebooks
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>